In [ ]:
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from collections import deque
from tiki.mini import TikiMini

# TikiMini 초기화
tiki = TikiMini()

# LED 제어 함수
def clear_leds():
    for i in range(16):
        tiki.set_led(0, i, 0, 0, 0)

def draw_shape(shape, color):
    clear_leds()
    r, g, b = color
    
    if shape == 'X':
        indices = [15, 0, 9, 6, 10, 5, 12, 3]
    elif shape == 'O':
        indices = [15, 8, 7, 0, 12, 11, 4, 3, 14, 13, 1, 2]
    elif shape == '#':
        indices = list(set([8, 9, 10, 11, 7, 6, 5, 4, 14, 9, 6, 1, 13, 10, 5, 2]))
    else:
        return
    
    for i in indices:
        tiki.set_led(0, i, r, g, b)

# QR 코드와 LED 패턴 매핑
qr_mapping = {
    'ID_O_R': ('O', (50, 0, 0)),
    'ID_O_G': ('O', (0, 50, 0)),
    'ID_O_B': ('O', (0, 0, 50)),
    'ID_X_R': ('X', (50, 0, 0)),
    'ID_X_G': ('X', (0, 50, 0)),
    'ID_X_B': ('X', (0, 0, 50)),
    'ID_#_R': ('#', (50, 0, 0)),
    'ID_#_G': ('#', (0, 50, 0)),
    'ID_#_B': ('#', (0, 0, 50)),
}

# CSI 카메라 파이프라인
pipeline = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)

if not cap.isOpened():
    pass
else:
    # 비디오 위젯 생성 (한 번만!)
    video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='640px', height='480px'))
    display(video_widget)
    
    def frame_to_bytes(frame):
        _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
        return buf.tobytes()
    
    qr_detector = cv2.QRCodeDetector()
    
    frame_count = 0
    last_qr_data = ""
    qr_detected = False
    
    try:
        while not qr_detected:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            frame = cv2.flip(frame, -1)
            
            # QR 코드 검출 (3프레임마다)
            data, bbox = None, None
            if frame_count % 3 == 0:
                data, bbox, _ = qr_detector.detectAndDecode(frame)
            
            if data:
                if data != last_qr_data:
                    last_qr_data = data
                    
                    if data in qr_mapping:
                        shape, color = qr_mapping[data]
                        draw_shape(shape, color)
                        qr_detected = True
                    else:
                        qr_detected = True
                
                # QR 코드 시각화 (인식했을 때만)
                if bbox is not None:
                    pts = bbox.astype(int).reshape(-1, 2)
                    for i in range(len(pts)):
                        cv2.line(frame, tuple(pts[i]), tuple(pts[(i + 1) % len(pts)]), (0, 255, 0), 2)
                    cv2.putText(frame, data, (pts[0][0], pts[0][1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                if last_qr_data != "":
                    last_qr_data = ""
            
            # 프레임 업데이트만 (clear_output, display 제거!)
            video_widget.value = frame_to_bytes(frame)
    
    except KeyboardInterrupt:
        cap.release()
        clear_leds()
    
    else:
        cap.release()